In [1]:
import numpy as np
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from neupy import algorithms
import pandas as pd
from scipy.optimize import basinhopping

# GRNN for all three output variables

In [6]:
class GRNN():
    def __init__(self, nn, scaler_x, scaler_y):
        self.nn = nn
        self.scaler_x = scaler_x
        self.scaler_y = scaler_y

def fit(X_, Y_):
    grnns = []
    for i in range(Y_.shape[1]):
        scaler_x = MinMaxScaler(feature_range=(0, 1))
        scaler_y = MinMaxScaler(feature_range=(0, 1))

        X = scaler_x.fit_transform(X_)
        Y = scaler_y.fit_transform(np.expand_dims(Y_[:, i], -1))

        x_train = X[:120, :]
        x_test = X[120:, :]

        y_train = np.expand_dims(Y[:120, :], -1)
        y_test = np.expand_dims(Y[120:, :], -1)
        
        nw = algorithms.GRNN(std=0.05, verbose=False)
        nw.train(x_train, y_train)
        
        grnns.append(GRNN(nw, scaler_x, scaler_y))
        
    return grnns

def calculate_error(grnns, X_, Y_):
    mses = []
    for i, grnn in enumerate(grnns):
        X = grnn.scaler_x.transform(X_)
        pred = grnn.nn.predict(X)
        pred= np.squeeze(pred, -1)
        pred = grnn.scaler_y.inverse_transform(pred)
        mse = np.mean((np.expand_dims(Y_[:, i], -1) - pred) ** 2)
        mses.append(mse)
    return mses

def predict(grnns, X, Y):
    tav = 0
    eff = 1
    tripple = 2
    preds = []
    for i, grnn in enumerate(grnns):
        x_scaled = grnn.scaler_x.transform(X)
        pred = grnn.nn.predict(x_scaled)
        pred = np.squeeze(pred, -1)
        pred = grnn.scaler_y.inverse_transform(pred)
        preds.append(pred)
        
    preds = np.concatenate(preds, axis=1)
    print(np.concatenate([Y, preds], axis=1))

X_ = pd.read_csv(r'C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\x.csv', header = None).values#[:, :]
Y_ = pd.read_csv(r'C:\Users\Saravanan\Desktop\maybe\paper work\syncrel\y.csv', header = None).values#[:, 0]
grnns = fit(X_, Y_)
x_test = X_[120:, :]
y_test = Y_[120:, :]
mses = calculate_error(grnns, x_test, y_test)
for mse in mses:
    print(mse)
predict(grnns, x_test, y_test)

9.738668440081812e-06
5.769308061080187e-09
0.04757969161972551
[[ 6.32143489  0.90482649 18.64073389  6.32105843  0.90473399 18.70804463]
 [ 6.48421538  0.90228284 15.79001894  6.47950469  0.90225164 16.00073574]
 [ 6.58771387  0.90212571 21.19043583  6.58835167  0.90205004 20.81801193]
 [ 6.76506092  0.89993901  8.88612263  6.76240479  0.89994831  9.00140122]
 [ 6.83610577  0.90034859 10.44046554  6.83740421  0.90025563 10.40055151]
 [ 7.02119884  0.89771078 14.91772976  7.02200764  0.89761107 14.55290893]
 [ 7.10372884  0.89819594 11.17074525  7.10340535  0.89808733 11.10024068]
 [ 7.28174719  0.89515915 12.66288722  7.27788328  0.89518852 12.28692472]
 [ 7.35485686  0.89525866 14.39108248  7.35272622  0.89516761 14.4961536 ]
 [ 7.51582938  0.89196143 17.82873233  7.51097968  0.89202736 17.67644505]
 [ 7.58544814  0.8931388  18.16916605  7.57980037  0.89307367 18.25412583]]
